# Binary search trees

A **binary search tree (BST)** is a type of binary tree in which each node contains a value and has at most two child nodes: a left child and a right child. Each node can be represented as a triplet $(l, v, r)$, where:

* $v$ is the value stored in the current node,
* $l$ is the left child,
* $r$ is the right child.

The tree satisfies the **binary search property**:
For every node, all values in its left subtree are strictly less than $v$, and all values in its right subtree are strictly greater than $v$:

$$
\forall x \in \text{left subtree of } v, \; x < v \quad \text{and} \quad \forall y \in \text{right subtree of } v, \; v < y
$$

This property must hold recursively for all nodes in the tree.

If elements are inserted into the tree in the random order, the average depth of nodes is $log(N)$ where $N$ is the number of nodes of the tree.

In [ ]:
from typing import Any

## Implementation

Consider an implementing of the tree using Python. Only a node that stores a value and refer to the two same nodes is needed.

In [25]:
class Node:
    def __init__(
        self,
        val: Any = 0,
        left: "Node | None" = None,
        right: "Node | None" = None
    ):
        self.val = val
        self.left = left
        self.right = right

Particular realisatoin of the tree example in the cell below:

In [26]:
#    [1]
#  [2] [3]
head = Node(
    1,
    Node(2),
    Node(3)
)

### Visualisation

Sometimes, it's useful to be able to visualize trees. To to visualize tree just in the standard output, we'll use the following format:

```
[Head node]
    [Left_1]
        [Left_l2]
        [Right_l2]
    [Right_1]
        [Left_r2]
        [Right_r2]
```

The follwing cell implements this function and displays the visualization outputs:

In [27]:
def print_tree(node: Node | None, indent: int = 0):
    if node is not None:
        print(f"{indent*' '}[{node.val}]")
        print_tree(node.left, indent+2)
        print_tree(node.right, indent+2)
    else:
        print(f"{indent*' '}[ ]")

head = Node(
    1,
    Node(
        2,
        Node(val=3),
        Node(val=4)
    ),
    Node(
        5,
        Node(val=6),
        Node(val=7)
    )
)
print_tree(head)

[1]
  [2]
    [3]
      [ ]
      [ ]
    [4]
      [ ]
      [ ]
  [5]
    [6]
      [ ]
      [ ]
    [7]
      [ ]
      [ ]
